# Capstone - Accident prediction


Name: Yahya Alhabboub

## Import Libraries

In [35]:
import pandas as pd
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

## Pre-processing

### Reading Dataset

In [36]:

df = pd.read_csv('/content/RTA Dataset (1).csv')

df

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,...,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,...,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,...,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,16:15:00,Wednesday,31-50,Male,NaN,Employee,2-5yr,Lorry (11?40Q),Owner,NaN,...,Going straight,na,na,na,na,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12312,18:00:00,Sunday,Unknown,Male,Elementary school,Employee,5-10yr,Automobile,Owner,NaN,...,Other,na,na,na,na,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12313,13:55:00,Sunday,Over 51,Male,Junior high school,Employee,5-10yr,Bajaj,Owner,2-5yrs,...,Other,Driver or rider,Male,31-50,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Serious Injury
12314,13:55:00,Sunday,18-30,Female,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,2-5yrs,...,Other,na,na,na,na,Driver,Normal,Not a Pedestrian,Driving under the influence of drugs,Slight Injury


### Data exploration

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12316 entries, 0 to 12315
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Time                         12316 non-null  object
 1   Day_of_week                  12316 non-null  object
 2   Age_band_of_driver           12316 non-null  object
 3   Sex_of_driver                12316 non-null  object
 4   Educational_level            11575 non-null  object
 5   Vehicle_driver_relation      11737 non-null  object
 6   Driving_experience           11487 non-null  object
 7   Type_of_vehicle              11366 non-null  object
 8   Owner_of_vehicle             11834 non-null  object
 9   Service_year_of_vehicle      8388 non-null   object
 10  Defect_of_vehicle            7889 non-null   object
 11  Area_accident_occured        12077 non-null  object
 12  Lanes_or_Medians             11931 non-null  object
 13  Road_allignment              12

### Missing Values ​​Declaration

In [38]:
df.replace("Unknown", np.nan, inplace=True)
df.replace("unknown", np.nan, inplace=True)
df.replace("na", np.nan, inplace=True)

In [39]:
df.isnull().sum()

,0
Time,0
Day_of_week,0
Age_band_of_driver,1548
Sex_of_driver,178
Educational_level,841
Vehicle_driver_relation,593
Driving_experience,862
Type_of_vehicle,950
Owner_of_vehicle,482
Service_year_of_vehicle,6811


### Print unique values

In [40]:
for i in df.columns:
  print(df[i].value_counts())
  print("null:" , df[i].isnull().sum())
  print("--------------------------------------")
  print("--------------------------------------")
  print("--------------------------------------")

Time
15:30:00    120
17:10:00    110
18:30:00    103
11:30:00     99
17:00:00     98
           ... 
0:52:00       1
4:49:00       1
14:31:00      1
19:18:00      1
2:47:00       1
Name: count, Length: 1074, dtype: int64
null: 0
--------------------------------------
--------------------------------------
--------------------------------------
Day_of_week
Friday       2041
Thursday     1851
Wednesday    1840
Tuesday      1770
Monday       1681
Saturday     1666
Sunday       1467
Name: count, dtype: int64
null: 0
--------------------------------------
--------------------------------------
--------------------------------------
Age_band_of_driver
18-30       4271
31-50       4087
Over 51     1585
Under 18     825
Name: count, dtype: int64
null: 1548
--------------------------------------
--------------------------------------
--------------------------------------
Sex_of_driver
Male      11437
Female      701
Name: count, dtype: int64
null: 178
--------------------------------------
---

### Feature Engineering

#### Drop undesirable columns

In [41]:
#  Drop undesirable columns
columns_to_drop = [
    'Service_year_of_vehicle',
    'Defect_of_vehicle',
    'Casualty_class',
    'Sex_of_casualty',
    'Age_band_of_casualty',
    'Casualty_severity',
    'Work_of_casuality',
    'Fitness_of_casuality'
]

df_cleaned = df.drop(columns=columns_to_drop)

#### Extract features

In [42]:
# Feature Engineering for the 'Time' column
# Convert 'Time' column to datetime object, then extract the hour
df_cleaned['Hour_of_day'] = pd.to_datetime(df_cleaned['Time'], format='%H:%M:%S', errors='coerce').dt.hour
df_cleaned = df_cleaned.drop(columns=['Time'])


#### Impute missing values

In [43]:
# Identify columns that still have missing values and need imputation
cols_with_missing_values = df_cleaned.columns[df_cleaned.isnull().any()]

print("Columns to be imputed:")
print(cols_with_missing_values)
print("-" * 30)

Columns to be imputed:
Index(['Age_band_of_driver', 'Sex_of_driver', 'Educational_level',
       'Vehicle_driver_relation', 'Driving_experience', 'Type_of_vehicle',
       'Owner_of_vehicle', 'Area_accident_occured', 'Lanes_or_Medians',
       'Road_allignment', 'Types_of_Junction', 'Road_surface_type',
       'Weather_conditions', 'Type_of_collision', 'Vehicle_movement',
       'Cause_of_accident'],
      dtype='object')
------------------------------


In [44]:
# Loop through each column and fill missing values with the mode
for col in cols_with_missing_values:
    mode_value = df_cleaned[col].mode()[0]
    df_cleaned[col].fillna(mode_value, inplace=True)
    print(f"Imputed missing values in '{col}' with '{mode_value}'")


Imputed missing values in 'Age_band_of_driver' with '18-30'
Imputed missing values in 'Sex_of_driver' with 'Male'
Imputed missing values in 'Educational_level' with 'Junior high school'
Imputed missing values in 'Vehicle_driver_relation' with 'Employee'
Imputed missing values in 'Driving_experience' with '5-10yr'
Imputed missing values in 'Type_of_vehicle' with 'Automobile'
Imputed missing values in 'Owner_of_vehicle' with 'Owner'
Imputed missing values in 'Area_accident_occured' with 'Other'
Imputed missing values in 'Lanes_or_Medians' with 'Two-way (divided with broken lines road marking)'
Imputed missing values in 'Road_allignment' with 'Tangent road with flat terrain'
Imputed missing values in 'Types_of_Junction' with 'Y Shape'
Imputed missing values in 'Road_surface_type' with 'Asphalt roads'
Imputed missing values in 'Weather_conditions' with 'Normal'
Imputed missing values in 'Type_of_collision' with 'Vehicle with vehicle collision'
Imputed missing values in 'Vehicle_movement' w

/tmp/ipython-input-2123746672.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(mode_value, inplace=True)


In [45]:
# Final check
print("-" * 30)
print("Total missing values after imputation:")
print(df_cleaned.isnull().sum())

------------------------------
Total missing values after imputation:
Day_of_week                    0
Age_band_of_driver             0
Sex_of_driver                  0
Educational_level              0
Vehicle_driver_relation        0
Driving_experience             0
Type_of_vehicle                0
Owner_of_vehicle               0
Area_accident_occured          0
Lanes_or_Medians               0
Road_allignment                0
Types_of_Junction              0
Road_surface_type              0
Road_surface_conditions        0
Light_conditions               0
Weather_conditions             0
Type_of_collision              0
Number_of_vehicles_involved    0
Number_of_casualties           0
Vehicle_movement               0
Pedestrian_movement            0
Cause_of_accident              0
Accident_severity              0
Hour_of_day                    0
dtype: int64


#### Split Data

In [46]:
# Prepare Target Variable and Initial Split

# Map target variable to numbers
severity_mapping = {'Slight Injury': 0, 'Serious Injury': 1, 'Fatal injury': 2}
df_cleaned['Accident_severity'] = df_cleaned['Accident_severity'].map(severity_mapping)

# Define features (X) and target (y)
X = df_cleaned.drop('Accident_severity', axis=1)
y = df_cleaned['Accident_severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y,shuffle= True)

#### Feature Engineering (Step 2)

In [47]:
# Feature Engineering

def feature_engineer(df):
    """Applies all feature engineering steps to a dataframe."""
    df_copy = df.copy() # Work on a copy to avoid SettingWithCopyWarning

    # a) Combine Rare Categories for 'Type_of_vehicle'
    # Calculate frequencies based on the training set only
    vehicle_counts = X_train['Type_of_vehicle'].value_counts(normalize=True)
    # Find categories that are less than 5% of the data
    rare_vehicles = vehicle_counts[vehicle_counts < 0.05].index
    # Replace rare categories with 'Other_Vehicle'
    df_copy['Type_of_vehicle'] = df_copy['Type_of_vehicle'].replace(rare_vehicles, 'Other_Vehicle')

    # b) Interaction Feature 1: Weather and Road Surface
    df_copy['Weather_Road_Interaction'] = df_copy['Weather_conditions'].astype(str) + '_' + df_copy['Road_surface_conditions'].astype(str)

    # c) Interaction Feature 2: Night with No Lights (Binary)
    night_no_light_condition = (df_copy['Hour_of_day'] >= 20) & (df_copy['Light_conditions'] == 'Darkness - no lighting')
    df_copy['Night_No_Light'] = night_no_light_condition.astype(int)

    return df_copy


X_train_eng = feature_engineer(X_train)
X_test_eng = feature_engineer(X_test)


#### Feature Encoding

In [48]:
target_encode_cols = [
    'Type_of_vehicle', 'Area_accident_occured', 'Cause_of_accident',
    'Weather_Road_Interaction', 'Lanes_or_Medians', 'Road_allignment', 'Types_of_Junction'
]

ohe_cols = [
    'Day_of_week', 'Sex_of_driver', 'Age_band_of_driver', 'Weather_conditions',
    'Road_surface_conditions', 'Light_conditions', 'Educational_level',
    'Vehicle_driver_relation', 'Driving_experience', 'Owner_of_vehicle', 'Road_surface_type','Type_of_collision','Pedestrian_movement','Vehicle_movement'
]

# a) Target Encoding
target_encoding_mappings = {}
global_target_mean = y_train.mean()
target_encoding_mappings['__global_mean__'] = global_target_mean

for col in target_encode_cols:
    target_mean_map = y_train.groupby(X_train_eng[col]).mean()
    target_encoding_mappings[col] = target_mean_map.to_dict()
    X_train_eng[col] = X_train_eng[col].map(target_mean_map)
    X_test_eng[col] = X_test_eng[col].map(target_mean_map)
    X_train_eng[col].fillna(global_target_mean, inplace=True)
    X_test_eng[col].fillna(global_target_mean, inplace=True)


# b) One-Hot Encoding
X_train_final = pd.get_dummies(X_train_eng, columns=ohe_cols, drop_first=True)
X_test_final = pd.get_dummies(X_test_eng, columns=ohe_cols, drop_first=True)

# Align columns to ensure consistency
train_cols = X_train_final.columns
test_cols = X_test_final.columns
missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_final[c] = 0
missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X_train_final[c] = 0
X_test_final = X_test_final[train_cols]


# -Display Final Results
print("--- Training Data Head (After FULL Engineering and Encoding) ---")
display_cols = [col for col in X_train_final.columns if 'Day_of_week' in str(col) or 'Educational_level' in str(col) or 'Lanes_or_Medians' in str(col)]
print(X_train_final[display_cols].head())

print(f"\nShape of final training data: {X_train_final.shape}")
print("All categorical columns should now be converted to numbers.")


/tmp/ipython-input-3330589923.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_eng[col].fillna(global_target_mean, inplace=True)
/tmp/ipython-input-3330589923.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

--- Training Data Head (After FULL Engineering and Encoding) ---
      Lanes_or_Medians  Day_of_week_Monday  Day_of_week_Saturday  \
3093          0.169328               False                 False   
4394          0.169328               False                 False   
584           0.169328               False                  True   
7724          0.169328               False                 False   
5532          0.169328               False                 False   

      Day_of_week_Sunday  Day_of_week_Thursday  Day_of_week_Tuesday  \
3093               False                 False                False   
4394                True                 False                False   
584                False                 False                False   
7724               False                 False                 True   
5532                True                 False                False   

      Day_of_week_Wednesday  Educational_level_Elementary school  \
3093                   True    

## Modeling

###  Create Pipeline

In [49]:
# 1. Create a pipeline to streamline the process
pipeline = Pipeline([
    ('classifier', RandomForestClassifier(random_state=42))
])

# 2. Define the parameter grid to search through
param_grid = [
    {
        'classifier': [RandomForestClassifier(random_state=42, class_weight='balanced')],
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20],
        'classifier__min_samples_leaf': [2, 4]
    },
    {
        'classifier': [GradientBoostingClassifier(random_state=42)],
        'classifier__n_estimators': [100, 200],
        'classifier__learning_rate': [0.05, 0.1],
        'classifier__max_depth': [3, 5]
    }
]


### Training the model on all features

In [50]:
# 3. Setup and run GridSearchCV

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train_final, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'classifier': [RandomForestClassifier(class_weight='balanced',
                                                                random_state=42)],
                          'classifier__max_depth': [10, 20],
                          'classifier__min_samples_leaf': [2, 4],
                          'classifier__n_estimators': [100, 200]},
                         {'classifier': [GradientBoostingClassifier(random_state=42)],
                          'classifier__learning_rate': [0.05, 0.1],
                          'classifier__max_depth': [3, 5],
                          'classifier__n_estimators': [100, 200]}],
             scoring='f1_weighted', verbose=1)

### Selecting best model




In [67]:
# Print the f1_weighted score for each model
print("--- F1 Weighted Scores for Each Model ---")
results = grid_search.cv_results_
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    print(f"Score: {mean_score:.4f} for params: {params}")

--- F1 Weighted Scores for Each Model ---
Score: 0.7903 for params: {'classifier': RandomForestClassifier(class_weight='balanced', random_state=42), 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 2, 'classifier__n_estimators': 100}
Score: 0.7906 for params: {'classifier': RandomForestClassifier(class_weight='balanced', random_state=42), 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 2, 'classifier__n_estimators': 200}
Score: 0.7829 for params: {'classifier': RandomForestClassifier(class_weight='balanced', random_state=42), 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__n_estimators': 100}
Score: 0.7855 for params: {'classifier': RandomForestClassifier(class_weight='balanced', random_state=42), 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__n_estimators': 200}
Score: 0.7868 for params: {'classifier': RandomForestClassifier(class_weight='balanced', random_state=42), 'classifier__max_depth': 20, 'class

In [51]:
# 4. Get the best model and print the results
print("--- Grid Search Results ---")
print(f"Best F1 Score (Weighted): {grid_search.best_score_:.4f}")
print("\nBest Model and Hyperparameters:")
print(grid_search.best_params_)

--- Grid Search Results ---
Best F1 Score (Weighted): 0.8046

Best Model and Hyperparameters:
{'classifier': GradientBoostingClassifier(random_state=42), 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 200}


#### Evaluate the best model on all features

In [52]:
# 5.1 Evaluate the best model on the train set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_train_final)
# Define target names for a readable report
target_names = ['Slight Injury (0)', 'Serious Injury (1)', 'Fatal injury (2)']

print("\n--- Classification Report for the Best Model on train Data ---")
print(classification_report(y_train, y_pred, target_names=target_names))


--- Classification Report for the Best Model on train Data ---
                    precision    recall  f1-score   support

 Slight Injury (0)       0.90      1.00      0.95      8331
Serious Injury (1)       0.99      0.35      0.52      1394
  Fatal injury (2)       1.00      0.89      0.94       127

          accuracy                           0.91      9852
         macro avg       0.96      0.75      0.80      9852
      weighted avg       0.92      0.91      0.89      9852



In [53]:
# 5.2 Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_final)
# Define target names for a readable report
target_names = ['Slight Injury (0)', 'Serious Injury (1)', 'Fatal injury (2)']

print("\n--- Classification Report for the Best Model on Test Data ---")
print(classification_report(y_test, y_pred, target_names=target_names))


--- Classification Report for the Best Model on Test Data ---
                    precision    recall  f1-score   support

 Slight Injury (0)       0.86      0.99      0.92      2084
Serious Injury (1)       0.60      0.10      0.17       349
  Fatal injury (2)       0.67      0.13      0.22        31

          accuracy                           0.85      2464
         macro avg       0.71      0.40      0.43      2464
      weighted avg       0.82      0.85      0.80      2464



### Select top 30 features

In [54]:
# 6. Extract and display the top 30 features from the best model
print("\n--- Top 30 Most Important Features ---")
# The actual classifier is in the 'classifier' step of the pipeline
final_classifier = best_model.named_steps['classifier']

# Get feature importances
importances = final_classifier.feature_importances_
feature_names = X_train_final.columns

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False).head(30)

print(feature_importance_df)


--- Top 30 Most Important Features ---
                                         feature  importance
8                                    Hour_of_day    0.118560
5                    Number_of_vehicles_involved    0.105171
6                           Number_of_casualties    0.073679
7                              Cause_of_accident    0.057985
4                              Types_of_Junction    0.055286
1                          Area_accident_occured    0.041625
0                                Type_of_vehicle    0.032985
9                       Weather_Road_Interaction    0.030941
2                               Lanes_or_Medians    0.028476
3                                Road_allignment    0.024858
33                     Light_conditions_Daylight    0.023052
16                         Day_of_week_Wednesday    0.021161
15                           Day_of_week_Tuesday    0.017604
12                          Day_of_week_Saturday    0.017387
13                            Day_of_week_Sun

### Training the model on top 30 features

In [61]:
# Get the list of top 30 feature names
top_30_features = feature_importance_df['feature'].tolist()

# Filter your training and testing data
X_train_top30 = X_train_final[top_30_features]
X_test_top30 = X_test_final[top_30_features]

# Retrain the BEST model on this smaller dataset
final_model = grid_search.best_estimator_ # This is already trained, so we can re-fit it
final_model.fit(X_train_top30, y_train)



Pipeline(steps=[('classifier',
                 GradientBoostingClassifier(max_depth=5, n_estimators=200,
                                            random_state=42))])

#### Evaluate the best model on top 30 features

In [62]:
print("\n--- Classification Report for the Best Model on train Data with Top 30 Features ---")
y_pred = final_model.predict(X_train_top30)
print(classification_report(y_train, y_pred, target_names=target_names))


--- Classification Report for the Best Model on train Data with Top 30 Features ---
                    precision    recall  f1-score   support

 Slight Injury (0)       0.90      1.00      0.95      8331
Serious Injury (1)       1.00      0.38      0.55      1394
  Fatal injury (2)       1.00      0.89      0.94       127

          accuracy                           0.91      9852
         macro avg       0.97      0.76      0.81      9852
      weighted avg       0.92      0.91      0.89      9852



In [63]:
print("\n--- Classification Report for the Best Model on Test Data with Top 30 Features ---")
y_pred_final = final_model.predict(X_test_top30)
print(classification_report(y_test, y_pred_final, target_names=target_names))


--- Classification Report for the Best Model on Test Data with Top 30 Features ---
                    precision    recall  f1-score   support

 Slight Injury (0)       0.86      0.99      0.92      2084
Serious Injury (1)       0.64      0.13      0.22       349
  Fatal injury (2)       0.50      0.16      0.24        31

          accuracy                           0.85      2464
         macro avg       0.67      0.43      0.46      2464
      weighted avg       0.83      0.85      0.81      2464



### Handle  imbalanced classes by SMOTE

In [64]:
# 1. Instantiate and apply SMOTE only on the training data
print("Class distribution before SMOTE:")
print(y_train.value_counts())

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_top30, y_train)

print("\nClass distribution after SMOTE:")
print(y_train_resampled.value_counts())


# 2. Retrain your best model on the new, balanced data
print("\nRetraining the best model on the resampled data...")
# Use the same winning model from GridSearchCV
final_model.fit(X_train_resampled, y_train_resampled)


Class distribution before SMOTE:
Accident_severity
0    8331
1    1394
2     127
Name: count, dtype: int64

Class distribution after SMOTE:
Accident_severity
0    8331
1    8331
2    8331
Name: count, dtype: int64

Retraining the best model on the resampled data...


Pipeline(steps=[('classifier',
                 GradientBoostingClassifier(max_depth=5, n_estimators=200,
                                            random_state=42))])

#### Evaluate the model after SMOTE

In [65]:
# 3.1 Evaluate the new model on the original, Train data
print("\n--- Final Classification Report After Using SMOTE ---")
y_pred_smote = final_model.predict(X_train_top30)
print(classification_report(y_train, y_pred_smote, target_names=target_names))


--- Final Classification Report After Using SMOTE ---
                    precision    recall  f1-score   support

 Slight Injury (0)       0.89      1.00      0.94      8331
Serious Injury (1)       0.95      0.26      0.41      1394
  Fatal injury (2)       0.95      0.75      0.84       127

          accuracy                           0.89      9852
         macro avg       0.93      0.67      0.73      9852
      weighted avg       0.90      0.89      0.86      9852



In [66]:
# 3.2 Evaluate the new model on the original, untouched test data
print("\n--- Final Classification Report After Using SMOTE on Test Data---")
y_pred_smote = final_model.predict(X_test_top30)
print(classification_report(y_test, y_pred_smote, target_names=target_names))


--- Final Classification Report After Using SMOTE on Test Data---
                    precision    recall  f1-score   support

 Slight Injury (0)       0.86      0.99      0.92      2084
Serious Injury (1)       0.58      0.08      0.15       349
  Fatal injury (2)       0.43      0.19      0.27        31

          accuracy                           0.85      2464
         macro avg       0.62      0.42      0.44      2464
      weighted avg       0.81      0.85      0.80      2464



## Key Insights

*   The best f1-score achieved was around 85%, and this result was similar across all the best performing models.
*   Reducing the dimensionality to 30 features and applying SMOTE did not significantly improve the model's performance.
*   The most important features were the hour of the day and details about the accident such as the number of vehicles involved, followed by weather and road conditions.